In [2]:
# app.py
from dotenv import load_dotenv
import os
from enum import Enum
from services.appleSetup import AppleAuth
from services.apple_maps import apple_maps_service
from services.google_maps import google_maps_service
from fastapi import FastAPI, Request, HTTPException, WebSocket, WebSocketDisconnect
from pydantic import BaseModel
from typing import List, Any, Optional, Dict
import appwrite
from appwrite.client import Client
from appwrite.query import Query
from appwrite.services.users import Users
from appwrite.services.databases import Databases
from appwrite.id import ID
from apscheduler.schedulers.background import BackgroundScheduler
from featureFunctions import get_search_region
from apscheduler.triggers.interval import IntervalTrigger
from datetime import datetime
import googlemaps
from appwrite.exception import AppwriteException
import httpx
import logging
from fastapi.middleware.cors import CORSMiddleware
from collections import defaultdict

# Installed
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
load_dotenv()

# Initialize Appwrite client
client = Client()
client.set_endpoint('https://cloud.appwrite.io/v1')
client.set_project('66930c61001b090ab206')
client.set_key(os.getenv('APPWRITE_API_KEY'))
client.set_self_signed()

appwrite_config = {
    "database_id": "66930e1000087eb0d4bd",
    "user_collection_id": "66930e5900107bc194dc",
    "preferences_collection_id": "6696016b00117bbf6352",
    "friends_collection_id": "friends",
    "locations_collection_id": "669d2a590010d4bf7d30",
    "groups_collection_id": "Groups",
    "contact_id": "66a419bb000fa8674a7e"
}

database = Databases(client)
users = Users(client)


Private key loaded successfully.
JWT generated successfully.
Centroid: {'lat': 35.999, 'lon': -118.60096666666668}
Search Region: 36.36131884057971,-118.15312162903858,35.63668115942029,-119.04881170429478


In [3]:
#1. We get all the preferences from each of the users that are passed in
#2: We get all the similar interests and the locations
#3: We query apple maps on the common interests and then 2-3 randomly selected ones, and then put the results in a array

def getPreferences(users: List[str]) -> Dict[str, Any]:
    res = defaultdict(int)
    for user in users:
        preferences = database.get_document(
            database_id=appwrite_config['database_id'],
            collection_id=appwrite_config['preferences_collection_id'],
            document_id=user
        )
        print(preferences)
        for key, value in preferences.items():
            if key == 'cuisine':
                for cuisine in value:
                    res[f"{cuisine} food"] += 1
            elif key == 'entertainment':
                for entertainment in value:
                    res[f"{entertainment}"] += 1
            elif key == 'shopping':
                if value == 'YES':
                    res['shopping'] += 1
            elif key == 'learning':
                for learning in value:
                    res[f"{learning}"] += 1
            elif key == 'sports':
                for sport in value:
                    res[f"{sport}"] += 1

    return res

# Example usage
preferences = getPreferences(["66996b7b0025b402922b", "66996d2e00314baa2a20", "669b45980030c00f3b8c", "669c735c001355ea24a7"])
print(preferences)

/opt/miniconda3/envs/headStarterBackend/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud.appwrite.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'user_id': '66996b7b0025b402922b', 'cuisine': ['Japanese', 'Italian'], 'atmosphere': ['Cozy'], 'entertainment': ['Arcade', 'Parks'], 'socializing': 'BOTH', 'Time': ['MORNING'], 'shopping': 'YES', 'family_friendly': True, 'learning': ['Culture'], 'sports': ['Golf'], '$id': '66996b7b0025b402922b', '$tenant': '168138', '$createdAt': '2024-07-18T19:31:11.173+00:00', '$updatedAt': '2024-07-20T19:16:56.252+00:00', '$permissions': [], 'users': {'email': 'Nagelia.naman@gmail.com', 'uid': '66996b7b0025b402922b', 'firstName': 'Naman', 'lastName': 'Nagelia', 'address': None, 'premium': False, 'monthly_uses': 5, 'friends': ['66996d2e00314baa2a20', '669ea8a0002fd0a07a97', '66a850af002730eadb39', '66a05a77002bacd6dad0', '66a58e55001acebf9f51'], 'receivedRequests': [], 'sentRequests': ['669b45980030c00f3b8c', '669c735c001355ea24a7'], '$id': '66996b7b0025b402922b', '$tenant': '168138', '$createdAt': '2024-07-18T19:22:37.018+00:00', '$updatedAt': '2024-07-31T18:48:29.871+00:00', '$permissions': [], '$

/opt/miniconda3/envs/headStarterBackend/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud.appwrite.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'user_id': '66996d2e00314baa2a20', 'cuisine': ['Indian', 'British'], 'atmosphere': ['Bustling'], 'entertainment': ['Eating', 'Beach'], 'socializing': 'BOTH', 'Time': ['MORNING', 'NIGHT'], 'shopping': 'YES', 'family_friendly': True, 'learning': ['Museums'], 'sports': ['Football'], '$id': '66996d2e00314baa2a20', '$tenant': '168138', '$createdAt': '2024-07-19T17:26:19.862+00:00', '$updatedAt': '2024-07-19T17:26:19.862+00:00', '$permissions': [], 'users': {'email': 'kzwke168@gmail.com', 'uid': '66996d2e00314baa2a20', 'firstName': 'Kasim', 'lastName': 'Zahid', 'address': 'England', 'premium': False, 'monthly_uses': 5, 'friends': ['66996b7b0025b402922b'], 'receivedRequests': [], 'sentRequests': ['66a05a77002bacd6dad0'], '$id': '66996d2e00314baa2a20', '$tenant': '168138', '$createdAt': '2024-07-18T19:29:51.884+00:00', '$updatedAt': '2024-07-28T19:01:26.885+00:00', '$permissions': [], '$databaseId': '66930e1000087eb0d4bd', '$collectionId': '66930e5900107bc194dc'}, '$databaseId': '66930e100008

/opt/miniconda3/envs/headStarterBackend/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud.appwrite.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'user_id': '669b45980030c00f3b8c', 'cuisine': ['Korean'], 'atmosphere': [], 'entertainment': ['Spas'], 'socializing': 'BOTH', 'Time': [], 'shopping': 'YES', 'family_friendly': False, 'learning': ['Culture'], 'sports': ['Go Kart'], '$id': '669b45980030c00f3b8c', '$tenant': '168138', '$createdAt': '2024-07-20T23:01:30.369+00:00', '$updatedAt': '2024-07-20T23:01:30.369+00:00', '$permissions': [], 'users': {'email': 'Ahmadbasyouni2004@gmail.com', 'uid': '669b45980030c00f3b8c', 'firstName': 'Ahmad', 'lastName': 'Basyouni', 'address': None, 'premium': False, 'monthly_uses': 5, 'friends': [], 'receivedRequests': ['66996b7b0025b402922b'], 'sentRequests': [], '$id': '669b45980030c00f3b8c', '$tenant': '168138', '$createdAt': '2024-07-20T05:05:29.656+00:00', '$updatedAt': '2024-07-28T03:58:23.477+00:00', '$permissions': [], '$databaseId': '66930e1000087eb0d4bd', '$collectionId': '66930e5900107bc194dc'}, '$databaseId': '66930e1000087eb0d4bd', '$collectionId': '6696016b00117bbf6352'}


/opt/miniconda3/envs/headStarterBackend/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud.appwrite.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'user_id': '669c735c001355ea24a7', 'cuisine': ['Mexican', 'Italian', 'Belgian', 'Pizza'], 'atmosphere': ['Lively'], 'entertainment': ['Arcade', 'Cinemas', 'Beach', 'Bars', 'Music', 'Theme Parks', 'Nightlife', 'Parks', 'Club', 'Spas'], 'socializing': 'ENERGETIC', 'Time': ['AFTERNOON', 'EVENING', 'NIGHT'], 'shopping': 'YES', 'family_friendly': False, 'learning': ['Historical Sites'], 'sports': ['Golf', 'Soccer', 'Aquatic Sports', 'Go Kart', 'Live Sports'], '$id': '669c735c001355ea24a7', '$tenant': '168138', '$createdAt': '2024-07-21T02:37:09.167+00:00', '$updatedAt': '2024-07-21T02:37:09.167+00:00', '$permissions': [], 'users': {'email': 'manas.nagelia@gmail.com', 'uid': '669c735c001355ea24a7', 'firstName': 'Manas', 'lastName': 'Nagelia', 'address': None, 'premium': False, 'monthly_uses': 5, 'friends': [], 'receivedRequests': ['66996b7b0025b402922b'], 'sentRequests': [], '$id': '669c735c001355ea24a7', '$tenant': '168138', '$createdAt': '2024-07-21T02:33:02.155+00:00', '$updatedAt': '202

In [21]:
class Location(BaseModel):
    lat: float
    lon: float

def calculate_centroid(locations: List[Location]) -> Dict[str, float]:
    """
    Calculate the centroid of given locations.
    """
    latitudes = [loc.lat for loc in locations]
    longitudes = [loc.lon for loc in locations]

    centroid_lat = sum(latitudes) / len(locations)
    centroid_lon = sum(longitudes) / len(locations)

    return {"lat": centroid_lat, "lon": centroid_lon}

#Get centroid with soem locations

locations = [Location(lat=38.98582939, lon= -76.937329584)]


In [22]:
import random
def getTopInterests(preferences: Dict[str, int], top_n: int = 10) -> List[str]:
    # Filter preferences with values greater than 2
    filtered_preferences = {k: v for k, v in preferences.items() if v > 2}
    
    # Sort the filtered preferences by value in descending order
    sorted_preferences = sorted(filtered_preferences.items(), key=lambda item: item[1], reverse=True)
    
    # Get the top N keys
    top_interests = [k for k, v in sorted_preferences[:top_n]]
    
    # If there are fewer than N keys, randomly select additional keys from the remaining preferences
    if len(top_interests) < top_n:
        remaining_preferences = {k: v for k, v in preferences.items() if k not in top_interests}
        additional_interests = random.sample(list(remaining_preferences.keys()), top_n - len(top_interests))
        top_interests.extend(additional_interests)
    
    return top_interests

interestsPassin = getTopInterests(preferences)
print(interestsPassin)


['shopping', 'Go Kart', 'Music', 'Parks', 'Live Sports', 'Culture', 'Japanese food', 'Italian food', 'Golf', 'Indian food']


In [23]:
#1. pass in the preferenes and location, sort preferences by volume and get the 10 most popular (if rest are ones randomly choose)
#2. then make the proximitiy request and get the list
class ProximityRecommendationRequest(BaseModel):
    locations: List[Location]
    interests: List[str]


async def get_proximity_recommendations(request: ProximityRecommendationRequest):
    """
    Generate recommendations based on the centroid of provided user locations.
    """
    centroid = calculate_centroid(request.locations)
    print(centroid)
    try:
        all_recommendations = []
        for interest in request.interests:
            results = await apple_maps_service.search(interest, centroid['lat'], centroid['lon'])
            for result in results:
                result['category2'] = interest
                if not result['category']:
                    result['category'] = interest  

            all_recommendations.extend(results)

        # Sort recommendations by distance from centroid (if needed)
        sorted_recommendations = sorted(
            all_recommendations,
            key=lambda x: ((x['location']['lat'] - centroid['lat'])**2 +
                           (x['location']['lon'] - centroid['lon'])**2)**0.5
        )

        # Limit to top N recommendations if needed
        top_recommendations = sorted_recommendations

        return {"recommendations": sorted_recommendations}
    except Exception as e:
        raise HTTPException(
            status_code=500, detail=f"Error getting recommendations: {str(e)}")

locs = await get_proximity_recommendations(ProximityRecommendationRequest(locations=locations, interests=interestsPassin))
print(locs)

{'lat': 38.98582939, 'lon': -76.937329584}


INFO:httpx:HTTP Request: GET https://maps-api.apple.com/v1/token "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://maps-api.apple.com/v1/search?q=shopping&searchLocation=38.98582939%2C-76.937329584 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://maps-api.apple.com/v1/token "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://maps-api.apple.com/v1/search?q=Go%20Kart&searchLocation=38.98582939%2C-76.937329584 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://maps-api.apple.com/v1/token "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://maps-api.apple.com/v1/search?q=Music&searchLocation=38.98582939%2C-76.937329584 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://maps-api.apple.com/v1/token "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://maps-api.apple.com/v1/search?q=Parks&searchLocation=38.98582939%2C-76.937329584 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://maps-api.apple.com/v1/token "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://m

{'recommendations': [{'name': 'Potomac Pizza', 'address': '7777 Baltimore Ave, College Park, MD  20740, United States', 'location': {'lat': 38.9873178, 'lon': -76.9356036}, 'category': 'Restaurant', 'category2': 'Italian food'}, {'name': 'Wasabi Bistro Japanese Food & Bubble Tea', 'address': '4505 College Ave, College Park, MD  20740, United States', 'location': {'lat': 38.9818553, 'lon': -76.9373256}, 'category': 'Restaurant', 'category2': 'Japanese food'}, {'name': 'Qù Japan', 'address': '7406 Baltimore Ave, College Park, MD  20740, United States', 'location': {'lat': 38.9811565, 'lon': -76.9380815}, 'category': 'Restaurant', 'category2': 'Japanese food'}, {'name': 'College Park Shopping Center', 'address': '7370 Baltimore Ave, College Park, MD  20740, United States', 'location': {'lat': 38.9806676, 'lon': -76.9390872}, 'category': 'Store', 'category2': 'shopping'}, {'name': 'The Spot Mini', 'address': '4207 Knox Rd, College Park, MD  20740, United States', 'location': {'lat': 38.981

In [24]:
import json
print(len(locs['recommendations']))

84


In [26]:
print(json.dumps(locs['recommendations'], indent=4))

[
    {
        "name": "Potomac Pizza",
        "address": "7777 Baltimore Ave, College Park, MD  20740, United States",
        "location": {
            "lat": 38.9873178,
            "lon": -76.9356036
        },
        "category": "Restaurant",
        "category2": "Italian food"
    },
    {
        "name": "Wasabi Bistro Japanese Food & Bubble Tea",
        "address": "4505 College Ave, College Park, MD  20740, United States",
        "location": {
            "lat": 38.9818553,
            "lon": -76.9373256
        },
        "category": "Restaurant",
        "category2": "Japanese food"
    },
    {
        "name": "Q\u00f9 Japan",
        "address": "7406 Baltimore Ave, College Park, MD  20740, United States",
        "location": {
            "lat": 38.9811565,
            "lon": -76.9380815
        },
        "category": "Restaurant",
        "category2": "Japanese food"
    },
    {
        "name": "College Park Shopping Center",
        "address": "7370 Baltimore Ave, C